Решение:

- Заместо парсинга будем решать задачу zero-shot классификации
- Для каждого класса подобрали самый лучший prompt
- Использовали CLIP модель (apple/DFN5B-CLIP-ViT-H-14-378), чтобы выбрать картинки близкие к каждому промтпу-тексту
- Класс "Остальное" назначали тем картинкам, у которых softmax threshold <= 0.2

Что ещё пробовали:

- Перебирали различные промпты
- Пробовали добавлять доп. классы, чтобы лучше отсеивать 16-ый класс
- Модели:
    - apple/DFN5B-CLIP-ViT-H-14-378
    - openai/clip-vit-large-patch14
    - openai/clip-vit-base-patch16
    - openai/clip-vit-large-patch14-336
    - openai/clip-vit-base-patch32
    - laion/CLIP-ViT-H-14-laion2B-s32B-b79K
    - google/siglip-so400m-patch14-384

Что ещё можно было попробовать:

- Кластеризация-классификация изображений с помощью ResNet с опорными изображениями
- Реально спарсить данные, но на многих сайтах они даны врассыпную и без категорий. Тогда всё равно пришлось бы использовать CLIP.
- Взять тренировочный датасет из [этого репозитория](https://github.com/omega-rg/Hotelify-A-classifier-for-hotel-images), там совпадают многие классы, но не ясно - улучшиться ли качество по сравнению с CLIP.

In [ ]:
import os

import pandas as pd
from transformers import pipeline

folder_path = "data/public_test"

class_labels = [
    "Hotel exterior, outdoor area, or building facade",
    "Hotel room, living space, or bedroom with furniture",
    "Swimming pool or hotel pool area",
    "Billiard table, pool table, or game room",
    "Bathroom with toilet, shower, sink, or bath amenities",
    "Hotel restaurant, dining room, or eating area",
    "Hotel lobby, reception area, or entrance hall",
    "Beachfront, shoreline, or sandy beach area",
    "Corridors, hallways, or staircases in the hotel",
    "Food dishes, meals on plates, or table settings",
    "Conference room, meeting room, or seminar space",
    "Gym, fitness center, or exercise equipment area",
    "Balcony view, outdoor balcony, or terrace",
    "Terrace, patio, or outdoor courtyard",
    "Spa, sauna, wellness center, or relaxation area",
]

image_classifier = pipeline(
    task="zero-shot-image-classification",
    model="apple/DFN5B-CLIP-ViT-H-14-378",
    device="cuda",
    batch_size=10,
)

image_files = [
    f"{folder_path}/{f}"
    for f in os.listdir(folder_path)
    if f.endswith(("jpg", "jpeg", "png"))
]
outputs = image_classifier(image_files, candidate_labels=class_labels)

/home/seara/Desktop/Github/mtc-classification/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
threshold = 0.2

new_outputs = [x[0] for x in outputs]
scores = [x["score"] for x in new_outputs]
labels = [x["label"] for x in new_outputs]
df = pd.DataFrame({"images": image_files, "labels": labels, "scores": scores})

df["class_number"] = df["labels"].apply(lambda x: class_labels.index(x)) + 1
df["class_number"] = df.apply(
    lambda x: x["class_number"] if x["scores"] > threshold else 16,
    axis=1,
)
df["image_name"] = df["images"].apply(lambda x: x.split("/")[-1])
df = df.sort_values(
    by="image_name", key=lambda x: [int(y.split("_")[2].split(".")[0]) for y in x]
)

df[["image_name", "class_number"]].to_csv(
    f"submissions/apple-DFN5B-CLIP-ViT-H-14-378_{threshold}_top_prompts_with_ts.csv",
    index=False,
)
df

,images,labels,scores,class_number,image_name
199,data/public_test/public_test_1.jpg,Swimming pool or hotel pool area,0.293851,3,public_test_1.jpg
281,data/public_test/public_test_2.jpg,"Hotel room, living space, or bedroom with furn...",0.297415,2,public_test_2.jpg
496,data/public_test/public_test_3.jpg,"Hotel room, living space, or bedroom with furn...",0.339467,2,public_test_3.jpg
379,data/public_test/public_test_4.jpg,"Gym, fitness center, or exercise equipment area",0.299372,12,public_test_4.jpg
458,data/public_test/public_test_5.jpg,Swimming pool or hotel pool area,0.239093,3,public_test_5.jpg
...,...,...,...,...,...
334,data/public_test/public_test_1120.jpg,"Conference room, meeting room, or seminar space",0.337431,11,public_test_1120.jpg
326,data/public_test/public_test_1121.jpg,"Gym, fitness center, or exercise equipment area",0.296985,12,public_test_1121.jpg
472,data/public_test/public_test_1122.jpg,"Hotel room, living space, or bedroom with furn...",0.246003,2,public_test_1122.jpg
394,data/public_test/public_test_1123.jpg,"Corridors, hallways, or staircases in the hotel",0.129925,16,public_test_1123.jpg
